<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-26 14:25:50
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.36 C
-------------------
Today PnL: -1.15 L (-0.84%)
Current PnL: -19.29 L (-13.12%)
CY Booked + Current PnL: -10.56 L (-7.18%)
-------------------
Total profit:  3.31 L
Total loss:  -22.60 L
-------------------
Total Booked + Current PnL: 16.42 L (13.68%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.42%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.40 L (57.63%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 7.89%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HINDUNILVR,2413.81,2723.00,-4.12,H-LC,8.23,301515.0,31168.0,3467.0,2.30,11.53,1.15,12.81,24.0,8.99,2.27,26.10,XY25,NTT,FMCG
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.34,112544.0,8232.0,8418.0,-1.74,7.89,7.48,15.96,9.0,0.98,0.85,26.12,X5K,ATH,METALS
55,MEDANTA,1087.93,1486.00,14.09,H-MC,14.71,162910.0,36710.0,9465.0,0.58,29.09,5.81,36.59,108.0,3.88,1.23,45.30,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-14.97,H-LC,8.08,169928.0,13751.0,12337.0,-1.62,8.80,7.26,16.71,22.0,1.11,1.28,27.78,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-13.25,M-LC,4.04,214836.0,13356.0,13857.0,-0.70,6.63,6.45,13.51,60.0,0.96,1.62,20.06,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,5.18,H-SC,11.06,125168.0,-129.0,59129.0,-2.61,-0.10,47.24,47.09,141.0,-0.00,0.94,56.66,MH,ATH,POWER
61,RAJOOENG,102.50,143.33,-31.64,H-SC,10.00,103510.0,1010.0,39820.0,-0.42,0.99,38.47,39.83,114.0,0.03,0.78,2.49,AR,ATH,MISC


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.19,H-LC,3.05,146565.0,-3367.0,17969.0,0.83,-2.25,12.26,9.74,4.0,-0.19,1.10,3.85,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-5.78,H-LC,3.21,186732.0,-13578.0,73554.0,-2.00,-6.78,39.39,29.94,16.0,-0.18,1.41,29.00,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.90,H-LC,3.40,274061.0,8635.0,48153.0,1.58,3.25,17.57,21.40,11.0,0.18,2.07,10.82,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-18.75,H-LC,4.21,216350.0,11850.0,29575.0,-1.37,5.79,13.67,20.25,10.0,0.40,1.63,15.03,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.54,H-LC,6.48,170612.0,-30774.0,118388.0,0.65,-15.28,69.39,43.51,27.0,-0.26,1.29,18.76,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SFL,1016.35,1287.0,43.20,M-SC,10.28,189075.0,-73143.0,142979.0,3.17,-27.89,75.62,26.63,229.0,-0.51,1.43,17.56,XY24,NTT,MISC
37,HINDUNILVR,2413.81,2723.0,-4.12,H-LC,8.23,301515.0,31168.0,3467.0,2.30,11.53,1.15,12.81,24.0,8.99,2.27,26.10,XY25,NTT,FMCG
30,GREENPANEL,375.16,537.0,195.87,M-SC,5.99,132077.0,-51001.0,129977.0,2.07,-27.86,98.41,43.14,230.0,-0.39,1.00,24.24,XY24,NTT,MISC
73,SONACOMS,578.05,1006.0,-35.37,M-SC,14.06,80404.0,-20755.0,95649.0,1.84,-20.52,118.96,74.03,202.0,-0.22,0.61,13.38,AR,BTT,AUTO
57,NESTLEIND,1134.30,1377.0,-10.90,H-LC,3.40,274061.0,8635.0,48153.0,1.58,3.25,17.57,21.40,11.0,0.18,2.07,10.82,XY25,NTT,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,214.25,400.0,-11.26,H-SC,8.48,191713.0,-56817.0,272290.0,-3.60,-22.86,142.03,86.70,151.0,-0.21,1.45,27.99,XY24,NTT,BANKS
88,VALIANTORG,512.64,838.0,-306.49,H-SC,13.11,87880.0,-45406.0,130001.0,-3.37,-34.07,147.93,63.47,139.0,-0.35,0.66,47.31,XR,NTT,CHEMICALS
13,BATAINDIA,1550.24,2096.0,-40.87,M-SC,9.05,88312.0,-40358.0,85654.0,-2.90,-31.37,96.99,35.20,219.0,-0.47,0.67,1.36,X40,NTT,FOOTWEAR
60,RAJESHEXPO,517.65,518.0,1627.27,L-SC,8.40,47120.0,-90057.0,90150.0,-2.85,-65.65,191.32,0.07,267.0,-1.00,0.36,17.19,OX40N,NTT,JEWELLERY
87,VAIBHAVGBL,350.21,521.0,36.64,H-SC,11.09,91075.0,-60916.0,135037.0,-2.69,-40.08,148.27,48.77,125.0,-0.45,0.69,6.01,XR,NTT,JEWELLERY


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,94.66,M-SC,8.63,83906.0,-16871.0,16966.0,-2.38,-16.74,20.22,0.09,245.0,-0.99,0.63,10.23,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,85.53,H-SC,15.47,116910.0,-25569.0,45092.0,-1.03,-17.95,38.57,13.70,163.0,-0.57,0.88,37.99,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.81,H-SC,3.04,114534.0,-29269.0,56019.0,-1.61,-20.35,48.91,18.60,149.0,-0.52,0.86,26.39,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,109.82,M-SC,1.69,137764.0,-36795.0,36824.0,-1.37,-21.08,26.73,0.02,221.0,-1.00,1.04,19.83,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.25,H-SC,3.64,211419.0,-58248.0,94589.0,0.16,-21.60,44.74,13.47,138.0,-0.62,1.59,6.93,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-25.80,H-LC,15.94,245895.0,-50039.0,143947.0,0.38,-16.91,58.54,31.73,1.0,-0.35,1.85,4.98,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.73,H-LC,11.87,272116.0,10515.0,135106.0,-0.78,4.02,49.65,55.67,3.0,0.08,2.05,10.37,X40,BTT,IT
47,ITC,411.90,452.00,-39.19,H-LC,3.05,146565.0,-3367.0,17969.0,0.83,-2.25,12.26,9.74,4.0,-0.19,1.10,3.85,X40,NTT,FMCG
89,VBL,492.64,671.64,-9.65,H-LC,8.64,274514.0,9474.0,86829.0,-1.11,3.57,31.63,36.33,5.0,0.11,2.07,17.35,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.10,H-LC,12.84,196443.0,-20076.0,112974.0,-1.24,-9.27,57.51,42.91,7.0,-0.18,1.48,2.88,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,14.09,H-MC,14.71,162910.0,36710.0,9465.0,0.58,29.09,5.81,36.59,108.0,3.88,1.23,45.30,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.11,H-LC,11.17,232064.0,47583.0,22789.0,0.23,25.79,9.82,38.14,38.0,2.09,1.75,39.13,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,14.09,H-MC,14.71,162910.0,36710.0,9465.0,0.58,29.09,5.81,36.59,108.0,3.88,1.23,45.30,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,8.71,H-MC,4.80,187700.0,23030.0,44935.0,-1.44,13.99,23.94,41.28,89.0,0.51,1.42,23.08,X40N,ATH,AC
90,VOLTAS,1278.28,1918.49,-3.32,H-MC,6.74,202950.0,11208.0,84833.0,-0.32,5.85,41.80,50.08,99.0,0.13,1.53,13.04,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,14.07,178658.0,27745.0,29568.0,0.80,18.38,16.55,37.97,93.0,0.94,1.35,48.34,AR,ATH,AUTO
25,DABUR,505.20,735.00,-6.00,H-MC,3.86,204750.0,7722.0,81900.0,1.23,3.92,40.00,45.49,102.0,0.09,1.54,17.47,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,14.07,178658.0,27745.0,29568.0,0.80,18.38,16.55,37.97,93.0,0.94,1.35,48.34,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.97,H-SC,12.37,139597.0,16261.0,117499.0,-0.14,13.18,84.17,108.45,119.0,0.14,1.05,38.48,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.94,M-SC,5.56,96390.0,4892.0,73864.0,-1.82,5.35,76.63,86.07,223.0,0.07,0.73,39.44,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-11.07,M-LC,7.77,155905.0,4960.0,104488.0,-0.94,3.29,67.02,72.51,53.0,0.05,1.18,9.51,XR,NTT,IT
61,RAJOOENG,102.50,143.33,-31.64,H-SC,10.00,103510.0,1010.0,39820.0,-0.42,0.99,38.47,39.83,114.0,0.03,0.78,2.49,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.07,82080.0,-31469.0,71516.0,-0.56,-27.71,87.13,35.27,268.0,-0.44,0.62,100.89,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.67,H-SC,16.15,89586.0,-11377.0,105909.0,-1.84,-11.27,118.22,93.63,148.0,-0.11,0.68,33.44,SR,ATH,CHEMICALS
54,MASFIN,326.60,398.61,-18.25,H-SC,12.56,93045.0,-4935.0,26536.0,-0.77,-5.04,28.52,22.05,152.0,-0.19,0.70,35.25,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.34,112544.0,8232.0,8418.0,-1.74,7.89,7.48,15.96,9.0,0.98,0.85,26.12,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,85.53,H-SC,15.47,116910.0,-25569.0,45092.0,-1.03,-17.95,38.57,13.70,163.0,-0.57,0.88,37.99,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.07,82080.0,-31469.0,71516.0,-0.56,-27.71,87.13,35.27,268.0,-0.44,0.62,100.89,XR,NTT,HOTELS
28,DMART,3764.92,5201.00,-7.11,H-LC,11.17,232064.0,47583.0,22789.0,0.23,25.79,9.82,38.14,38.0,2.09,1.75,39.13,X40N,NTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,14.07,178658.0,27745.0,29568.0,0.80,18.38,16.55,37.97,93.0,0.94,1.35,48.34,AR,ATH,AUTO
69,SFL,1016.35,1287.00,43.20,M-SC,10.28,189075.0,-73143.0,142979.0,3.17,-27.89,75.62,26.63,229.0,-0.51,1.43,17.56,XY24,NTT,MISC
37,HINDUNILVR,2413.81,2723.00,-4.12,H-LC,8.23,301515.0,31168.0,3467.0,2.30,11.53,1.15,12.81,24.0,8.99,2.27,26.10,XY25,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.04
1,25,41.66
2,50,71.35


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.19
LC    36.48
MC    26.35
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.40
X40      14.46
X40N     13.94
XY25     11.94
XR        8.98
AR        8.47
OX40N     7.36
X5K       2.26
MH        1.69
X200      1.41
SR        1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.24
H-MC    23.47
H-SC    22.43
M-SC    13.21
M-LC     7.19
M-MC     2.55
L-SC     1.55
L-LC     1.05
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.72,0.33,30.20
IT,12.06,-16.73,75.03
FINANCE,7.98,-23.16,68.70
BANKS,7.44,-15.70,64.83
MISC,5.96,-16.38,83.09
PAINTS,5.53,-19.07,42.53
ELECTRICAL,4.73,-12.00,60.50
AUTO,4.60,-11.08,59.41
AC,3.68,0.80,32.80


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2869346.0,23
XR,1104149.0,13
AR,1085923.0,10
X40,817473.0,11
X40N,518131.0,11
OX40N,481259.0,10
XY25,463709.0,8
SR,196967.0,2
X5K,138749.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2798530.0,27
M-SC,1493231.0,17
H-MC,1267100.0,18
H-LC,1155141.0,19
M-LC,425496.0,5
M-MC,339113.0,2
L-SC,256705.0,3
L-MC,59486.0,1
L-LC,45372.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      947284.0      6
M-SC       XY24      869320.0      7
H-SC       AR        663164.0      5
           XR        530984.0      6
H-LC       X40       511231.0      6
H-MC       XY24      486960.0      5
M-MC       XY24      339113.0      2
H-LC       AR        234967.0      2
H-MC       X40       220588.0      4
H-SC       OX40N     206953.0      4
H-MC       XY25      199668.0      2
H-SC       SR        196967.0      2
H-LC       X40N      196269.0      5
H-SC       X40N      194049.0      3
M-LC       XY24      176820.0      2
L-SC       XR        166555.0      2
M-SC       AR        158224.0      2
           OX40N     132542.0      4
M-LC       X5K       130331.0      1
H-MC       X40N      127813.0      3
M-SC       XY25      123959.0      1
           XR        123532.0      2
H-MC       XR        119104.0      1
M-LC       XR        104488.0      1
L-SC       OX40N      90150.0      1
M-SC       X40        85654.0      1
H-LC       XY25       80853.0      3
           X200       73554.0      1
L-MC       XR         59486.0      1
H-SC       MH         59129.0      1
H-MC       OX40N      51614.0      1
H-LC       XY24       49849.0      1
L-LC       XY25       45372.0      1
H-MC       MH         31785.0      1
           AR         29568.0      1
M-LC       XY25       13857.0      1
H-LC       X5K         8418.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
